# Import Libraries

In [1]:
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader
import os
import faiss

/workspaces/digital_research_guide/phi3_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [2]:
import pre_processing

# Load Embedding Model

In [3]:
check_point = 'mixedbread-ai/mxbai-embed-large-v1'
embedding_model = SentenceTransformer(check_point)

/workspaces/digital_research_guide/phi3_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/workspaces/digital_research_guide/phi3_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
pre_processing.model = embedding_model

# Process Doc

In [5]:
reader = PdfReader('/workspaces/digital_research_guide/1706.03762v7.pdf')

In [6]:
context_list = pre_processing.parese_doc(reader)
pre_processing.create_embedding(context_list)

# Linking ONXX Model

In [7]:
import onnxruntime_genai as og

In [8]:
model = og.Model('/workspaces/digital_research_guide/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4')
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

In [9]:
chat_template = '<|user|>\n{input} <|end|>\n<|assistant|>'

In [10]:
search_options ={}
#search_options['max_length'] = 4000
search_options['temperature'] = 1

In [11]:
base_path = os.getcwd()

In [12]:
index: faiss.IndexFlatL2 = faiss.read_index(os.path.join(base_path, 'doc.index'))

In [13]:
while True:
        text = input("Input: ")
        if not text:
            print("Error, input cannot be empty")
            break

        # If there is a chat template, use it
        #prompt = f'{chat_template.format(input=text)}'



        query_embedding = embedding_model.encode(text).reshape(1, -1)
        top_k = 1
        _scores, binary_ids = index.search(query_embedding, top_k)
        binary_ids = binary_ids[0]
        _scores = _scores[0]
        temp_list = []
        for idx in binary_ids:
             temp_list.append(context_list[idx])
        context = '. '.join(temp_list)
        
        text += " With respect to context: "+context
        

        prompt = f'{chat_template.format(input=text)}'
        print(prompt)



        input_tokens = tokenizer.encode(prompt)

        params = og.GeneratorParams(model)
        params.try_use_cuda_graph_with_max_batch_size(1)
        params.set_search_options(**search_options)
        params.input_ids = input_tokens
        generator = og.Generator(model, params)
        

        print()
        print("Output: ", end='', flush=True)

        try:
            while not generator.is_done():
                generator.compute_logits()
                generator.generate_next_token()
                new_token = generator.get_next_tokens()[0]
                print(tokenizer_stream.decode(new_token), end='', flush=True)
        except KeyboardInterrupt:
            print("  --control+c pressed, aborting generation--")
        print()
        print()

<|user|>
what is transformers With respect to context: Llion also experimented with novel model variants, was responsible for our initial codebase, and efficient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and implementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating our research. †Work performed while at Google Brain. ‡Work performed while at Google Research. 31st Conference on Neural Information Processing Systems (NIPS 2017), Long Beach, CA, USA.arXiv:1706.03762v7  [cs.CL]  2 Aug 20231 Introduction Recurrent neural networks, long short-term memory [ 13] and gated recurrent [ 7] neural networks in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems such as language modeling and machine translation [ 35,2,5]. Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-